In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
url = 'https://www.jiomart.com/c/groceries/2?prod_mart_groceries_products_popularity%5Bpage%5D=2'

In [23]:
response = requests.get(url)

In [24]:
response.status_code

200

In [25]:
soup = BeautifulSoup(response.content,'html.parser')

In [31]:
results = soup.find_all('div',{'class':'plp-card-details-container'})

In [32]:
len(results)

12

In [36]:
#title
results[0].find('div',{'class':'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'}).get_text().strip()

'Super Sarvottam Physically Refined Rice Bran Oil 1 L'

In [37]:
# price
results[0].find('span',{'class':'jm-heading-xxs jm-mb-xxs'}).get_text().strip()

'₹99.00'

In [41]:
#discount
results[0].find('span',{'class':'jm-badge'}).get_text().strip()

'50% OFF'

In [88]:
# Puting Everything in loop

product_name = []
product_price = []
product_discount = []
for i in range(len(results)):
    badge_element = results[i].find('span', {'class': 'jm-badge'})
    product_name.append(results[i].find('div',{'class':'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'}).get_text().strip())
    product_price.append(results[i].find('span',{'class':'jm-heading-xxs jm-mb-xxs'}).get_text().strip())
    if badge_element is not None:
        text_inside_badge = badge_element.get_text().strip()
        if text_inside_badge:  # Check if the text is not an empty string
            product_discount.append(text_inside_badge)
        else:
            # Handle the case when the text is empty, e.g., set a default value
            product_discount.append("None")  # Replace "No Discount" with your desired default value
    else:
        # Handle the case when the element is not found, e.g., set a default value
        product_discount.append("None")

In [91]:
product_name

['Super Sarvottam Physically Refined Rice Bran Oil 1 L',
 'Closeup Everfresh+ Red Hot Gel Toothpaste 150 g (Pack of 2)',
 'Surf Excel Easy Wash Detergent Powder 5 kg',
 'Priya Fortified With Vitamin A & D Refined Sunflower Oil 1 L',
 'Bharat Chana Dal 1 Kg',
 'Pears Pure & Gentle Soap with Natural Oils 125 g (Buy 4 Get 1 Free)',
 'Maaza Mango Drink 1.2 L',
 'Surf Excel Detergent Bar 200 g (Pack of 4)',
 'Rin Detergent Bar 110 g',
 'Kissan Fresh Tomato Ketchup 1.1 kg',
 'Tata Iodised Salt 1 kg',
 'Maggi 2-Minute Masala Noodles 70 g']

In [92]:
product_price

['₹99.00',
 '₹154.00',
 '₹678.00',
 '₹96.00',
 '₹60.00',
 '₹250.00',
 '₹50.00',
 '₹112.00',
 '₹9.00',
 '₹110.00',
 '₹24.00',
 '₹13.00']

In [120]:
product_discount

['50% OFF',
 '33% OFF',
 '7% OFF',
 '40% OFF',
 'None',
 '33% OFF',
 '33% OFF',
 '8% OFF',
 '10% OFF',
 '26% OFF',
 '14% OFF',
 '7% OFF']

In [162]:
data = {'Product Name':product_name,'Price':product_price,'Discount':product_discount}
df = pd.DataFrame(data)


In [163]:
df['Discount'] = df['Discount'].replace('None','0% OFF')

In [164]:
df

,Product Name,Price,Discount
0,Super Sarvottam Physically Refined Rice Bran O...,₹99.00,50% OFF
1,Closeup Everfresh+ Red Hot Gel Toothpaste 150 ...,₹154.00,33% OFF
2,Surf Excel Easy Wash Detergent Powder 5 kg,₹678.00,7% OFF
3,Priya Fortified With Vitamin A & D Refined Sun...,₹96.00,40% OFF
4,Bharat Chana Dal 1 Kg,₹60.00,0% OFF
5,Pears Pure & Gentle Soap with Natural Oils 125...,₹250.00,33% OFF
6,Maaza Mango Drink 1.2 L,₹50.00,33% OFF
7,Surf Excel Detergent Bar 200 g (Pack of 4),₹112.00,8% OFF
8,Rin Detergent Bar 110 g,₹9.00,10% OFF
9,Kissan Fresh Tomato Ketchup 1.1 kg,₹110.00,26% OFF


In [165]:
# df['Original Price'] = df['Price']+ (df['Price']*0.5)
df['Price'] = df['Price'].str[1:]

In [166]:
df['Price']=df['Price'].astype(float)

In [167]:
df['Discount'] = df['Discount'].str[0:-5]

In [170]:
df['Discount'] = df['Discount'].astype(float)

In [173]:
df['Original Price'] = df['Price'] / (1 - df['Discount']/100)

In [174]:
df['Original Price'] = round(df['Original Price'])

In [175]:
df

,Product Name,Price,Discount,Original Price
0,Super Sarvottam Physically Refined Rice Bran O...,99.0,50.0,198.0
1,Closeup Everfresh+ Red Hot Gel Toothpaste 150 ...,154.0,33.0,230.0
2,Surf Excel Easy Wash Detergent Powder 5 kg,678.0,7.0,729.0
3,Priya Fortified With Vitamin A & D Refined Sun...,96.0,40.0,160.0
4,Bharat Chana Dal 1 Kg,60.0,0.0,60.0
5,Pears Pure & Gentle Soap with Natural Oils 125...,250.0,33.0,373.0
6,Maaza Mango Drink 1.2 L,50.0,33.0,75.0
7,Surf Excel Detergent Bar 200 g (Pack of 4),112.0,8.0,122.0
8,Rin Detergent Bar 110 g,9.0,10.0,10.0
9,Kissan Fresh Tomato Ketchup 1.1 kg,110.0,26.0,149.0


In [176]:
df.to_csv('Jio_Mart.csv')